In [ ]:
!pip install beautifulsoup4

In [ ]:
# Se certifique que você está utilizando Python 3 e utilizando UTF-8
print(sys.getdefaultencoding())
print(sys.version)

In [ ]:
# Google Scholar Crawler
# Para executar o script substitua o filename com o nome do arquivo que contém as URLs de busca
#
# -*- coding: utf-8 -*-

# Próximas etapas: 
# - Começar a baixar os PDFs

import requests
import random
import time
from bs4 import BeautifulSoup

last_url = ''
last_soup = ''
last_author = ''
PROXY_LIST = []

USER_AGENT_CHOICES = [
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:23.0) Gecko/20100101 Firefox/23.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.62 Safari/537.36',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.146 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.146 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64; rv:24.0) Gecko/20140205 Firefox/24.0 Iceweasel/24.3.0',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:28.0) Gecko/20100101 Firefox/28.0',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:28.0) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
]

from lxml.html import fromstring
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    headers = {'user-agent': random.choice(USER_AGENT_CHOICES) }
    response = requests.get(url,  headers=headers)
    parser = fromstring(response.text)
    lista = []
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            lista.append(proxy)
    print('Proxies obtidos: %d' % len(lista))
    return lista

def get_page(url):
    global PROXY_LIST
    last_soup = ''
    headers = {'user-agent': random.choice(USER_AGENT_CHOICES) }
    while len(PROXY_LIST) > 0:
        try:
            proxy = random.choice(PROXY_LIST)
            response = requests.get(url, headers=headers, proxies={ "http": proxy, "https": proxy, },timeout=30)
            if response.status_code == 200:
                print('%s (%s)' % (url,proxy))
                if u'Ative o JavaScript' in response.text:
                    print('Proxy Blocked 1')
                    raise Exception('Blocked')
                last_soup = BeautifulSoup(response.text, 'html.parser')
                break
            else:
                print('Proxy Blocked 2')
                raise Exception('Blocked')
        except:
            PROXY_LIST.remove(proxy)
            print('Dead Proxy: %s - Left: %d' % (proxy, len(PROXY_LIST)) )
            while len(PROXY_LIST) == 0:
                time.sleep(60)
                PROXY_LIST = get_proxies()
            
    return last_soup

def find_artigos(soup):
    try:
        lista = soup.find('div', attrs={'id': 'gs_bdy'}).find('div', attrs={'id': 'gs_bdy_ccl'})
        lista = lista.find('div',attrs={'id': 'gs_res_ccl'})
        lista = lista.find('div',attrs={'id': 'gs_res_ccl_mid'})
    except:
        if soup:
            print('erro:'+soup.text)
        lista = None
    return lista

def parse_pagina(lista, artigos, origem):
    incluidos = 0
    for registro in lista.findAll('div',attrs={'class':'gs_r gs_or gs_scl'}):

        tipo = registro.find('div',attrs={'class':'gs_or_ggsm'})
        if tipo:
            tipo = tipo.find('span').text
        else:
            tipo = 'HTML'
            
        artigo = { 'tipo': tipo, 'citado': 0 } # Contem a indicação se é HTML ou PDF
        
        item = registro.find('div',attrs={'class':'gs_ri'})
        divs = item.findAll('div')
        titulo = item.find('h3')
        if titulo and titulo.find('a'):
            artigo['url'] = titulo.find('a')['href']
            artigo['titulo'] =  titulo.find('a').text
        else:
            artigo['titulo'] = 'Titulo não encontrado (%d)' % incluidos
            artigo['url'] = last_url

        detalhe = divs[0].text.split('- ')
        autores = {}
        for autor in detalhe[0].split(','):
            nome = autor.strip()
            autores[ nome ] = ''
        
        for alink in divs[0].findAll('a'):
            nome = alink.text.split('- ')[0].strip()
            autores[ nome ] = alink['href'].split('&')[0]
    
        #for autor in sorted(autores.keys()):
        #    print('Autor :%s' % autor)
        #    print('Link  :%s' % autores[autor])
               
        artigo['autores'] = autores

        year = ''
        journal = ''
        if len(detalhe) > 1:
            journal = detalhe[1].strip()
            if len(journal.split(',')) > 1:
                year = journal.rsplit(',',1)[1].strip()
                journal = journal.split(',')[0].strip()
            else:
                if len(journal) == 4:
                    year = journal
                    journal = ''
                
        artigo['journal'] = journal
        artigo['year'] = year
        artigo['origem'] = origem

        try:
            item = divs[2]
            for celula in item.findAll('a'):
                if celula.text[0:3].upper() == 'CIT' and len(celula.text.split(' ')) > 1:   # Cited by ou citado por
                    artigo['citado'] = celula.text.split(' ')[2]
        except:
            artigo['citado'] = 0
        
        artigos.append( artigo )
        incluidos += 1
    return incluidos

# Monta a lista de URLs que serão utilizadas
#
# exemplos de url que busca um artigo determinado:
# https://scholar.google.com.br/scholar?hl=pt-BR&lr=lang_pt&as_sdt=0,5&q=label%3A+digital+humanities
#
# exemplos de url que cita um determinado artigo e que tem o retorno específico em português:
# https://scholar.google.com/scholar?cites=537519748848001123&as_sdt=2005&sciodt=0,5&lr=lang_pt&hl=pt-BR'
# https://scholar.google.com.br/scholar?lr=lang_pt&hl=pt-BR&as_sdt=0,5&sciodt=0,5&cites=17950367634186489359,349589428917357430,1179427091885643356

urls = []       
filename = 'cesta2.txt'
if not os.path.isfile(filename):
    raise Exception('Arquivo %s não encontrado' % filename)
cesta = open(filename, 'r')
for linha in cesta:
    urls.append( linha.strip() )
cesta.close()

# Monta lista de Proxies
PROXY_LIST = get_proxies()
if len(PROXY_LIST) == 0:
    raise Exception('No proxies found')

# realizar o scraping de cada URL e armazenar os artigos encontrados na lista artigos
# a rotina armazena também quantos registros foram obtidos em cada consulta
url_index = 1
artigos = []
resultado_por_url = []
for url_base in urls:
    total = 0
    incluidos = 0 
    while True:
        last_url = url_base+'&as_vis=1&start=%d' % total
        soup = get_page( last_url )
        lista = find_artigos(soup)
        if lista:
            incluidos = parse_pagina(lista, artigos, url_index)
            total += incluidos
            if incluidos < 10:
                break
    # necessário para que não hajam muitas requisições instantâneas para que o Google não bloqueie mais
    # rapidamente as consultas
    time.sleep(10)
    url_index += 1
    resultado_por_url.append(total)
print('Fim')

In [ ]:
print(len(artigos))

In [ ]:
numero = 0
for artigo in artigos:
    titulo = artigo['titulo']
    print(numero, titulo, artigo['autores'])
    numero += 1

In [ ]:
# Removendo as URLs que tiveram resultado nulo
# Essa rotina só é necessária para otimizar o processo, fazendo com que a rotina realize menos consultas ao GS
import csv
index = 0
gravados = 0
with open('cesta2.txt', 'w') as arquivo:
    for url in urls:
        if len(resultado_por_url) <= index or resultado_por_url[ index ] > 0:
            arquivo.write(url+'\n')
            gravados += 1
        index += 1
    arquivo.close()
print('Lidos %d' % index)
print('Gravados %d' % gravados)

In [ ]:
# Grava 
import csv
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['titulo','autores','ano','citacoes','tipo','url','origem'])
    for artigo in artigos:
        if 'citado' in artigo:
            citacoes = artigo['citado']
        else:
            citacoes = 0
        writer.writerow([artigo['titulo'],artigo['autores'],artigo['year'],citacoes,artigo['tipo'],artigo['url'],artigo['origem']])
print('arquivo gravado com sucesso!')

In [ ]:
# Cesta Final contém as referências das URLs pesquisadas
# Necessário para que se possa descobrir qual a busca que recuperou o arquivo (Campo Origem)
import csv
with open('cestafinal.txt', 'w') as arquivo:
    for url in urls:
        arquivo.write(url+'\n')
    arquivo.close()

In [ ]:
# Retira os registros duplicados
# Corrige também os registros que não tiveram o campo 'citado' registrado nos casos em que ele não foi encontrado
# Remove artigos cuja origem estava errada

base = set()
dedup = []
for d in artigos:
    if d['origem'] != 56:
        t = tuple(d['titulo'].strip())
        if t not in base:
            base.add(t)
            dedup.append(d)

print(len(dedup))

import csv
with open('dedup.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(['titulo','autores','ano','citacoes','tipo','url','origem'])
    for artigo in dedup:
        if 'citado' in artigo:
            citacoes = artigo['citado']
        else:
            citacoes = 0
        writer.writerow([artigo['titulo'],artigo['autores'],artigo['year'],citacoes,artigo['tipo'],artigo['url'],artigo['origem']])
print('arquivo gravado com sucesso!')

In [ ]:
# Relação de autores únicos

from collections import Counter

totais = Counter()
autores = {}
for reg in dedup:
    for autor, link in reg['autores'].items():
        autor_normalizado = autor.replace('…','').strip().upper()
        if autor_normalizado in autores:
            if autores[autor_normalizado] == '' and link != '':
                autores[autor_normalizado] = link
            if autores[autor_normalizado] != link:
                print('different URL: %s %s %s' % (autor_normalizado,autores[autor_normalizado],link))
        else:
            autores[autor_normalizado] = link
        totais[autor_normalizado] += 1
            
print(len(autores))
freqs = 0
for top in totais.most_common():
    if top[1] > 1:
        print(top)
        freqs +=1 
print(freqs)

In [ ]:
# Artigos da Cesta com mais citações

from collections import Counter

totais = Counter()
for reg in dedup:
    totais[ reg['origem'] ] += 1

print(len(totais))
for top in totais.most_common():
    index = top[0] - 1
    if index >= len(urls):
        print('Indice fora da faixa! %d' % index )
    else:
        print(urls[ index ], top[1])

In [ ]:
# Relação de autores com mais co-autoria

from collections import Counter

totais = Counter()
coautorias = {}
for reg in dedup:
    if len(reg['autores']) > 1:
        coautoria = ''
        for autor, link in reg['autores'].items():
            coautoria = coautoria + autor.replace('…','').strip().upper()+','
        coautoria = coautoria[:-1]
        totais[coautoria] += 1

print(len(totais))
for top in totais.most_common():
    print(top)